In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import PIL
import PIL.Image
import numpy as np


In [2]:
force_path = '../force_data/'
timestamp_path = '../image_timestamp/'

In [3]:
stamp_force = pd.read_csv(force_path+'sync-test-1.csv') 
stamp_frame = pd.read_csv(timestamp_path+'sync-test-1.csv') 

In [4]:
frame_force = {}
for i in range(len(stamp_force)):
    frame_force[stamp_frame.iloc[i]['frame']] = stamp_force.iloc[i]['force']

In [5]:
data_dir = pathlib.Path('../small_imgs/')
images = list(data_dir.glob('*.png'))

In [6]:
pics = []
forces = []

for i in range(len(images)):
    if i in frame_force.keys():
        I = np.array(PIL.Image.open(str(images[i])))
        img = PIL.Image.fromarray(I, 'RGB')
        img = np.asanyarray(img)
        pics.append(img)
        forces.append(frame_force[i])


In [7]:
event_frames = []
event_forces = []
event_index = []
for i in range(1, len(forces)):
    if forces[i] != 0:
        event_frames.append(pics[i])
        event_forces.append(forces[i])
        event_index.append(i)
    else:
        if forces[i-1] != 0:
            event_frames.append(pics[i])
            event_forces.append(forces[i])
            event_index.append(i)

In [8]:
events = []
current_event = []

for i in event_index:
    if forces[i] == 0:
        events.append(current_event)
        current_event = []
    else:
        current_event.append(i)
events[0] = [0]+events[0]        

In [9]:
x = []
y = []
for e in events:
    s = e[0]
    if len(e) < 40:
        new_s = s - (40-len(e))//2
    else:
        new_s = s
    x.append(pics[new_s:new_s+40])
    y.append(min(forces[new_s:new_s+40]))

#### Model construction
* Create a bath of scenes for training
* Each scene is a series of pictures representing one event (a probe touching an object)
* Input simension is (# of scenes, # of frames in a scene, height of picture, width of picture, # of color channels)
* Use a time distributed CNN to process each frame in a scene
* Pass each scene as an array of frames to RNN, so that each sample is a scene with a corresponding force number

In [10]:
x = np.array(x)
y = np.array(y)
x = x / 255.0

In [11]:
x.shape

(5, 40, 72, 128, 3)

In [23]:
model = models.Sequential()
# CNN part
model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu')))

# Flatten CNN output
model.add(layers.TimeDistributed(layers.Flatten()))

# RNN part

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# # final dense layers for output
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))
model.build(input_shape = x.shape)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_6 (TimeDist (5, 40, 70, 126, 64)      1792      
_________________________________________________________________
time_distributed_7 (TimeDist (5, 40, 35, 63, 64)       0         
_________________________________________________________________
time_distributed_8 (TimeDist (5, 40, 33, 61, 64)       36928     
_________________________________________________________________
time_distributed_9 (TimeDist (5, 40, 16, 30, 64)       0         
_________________________________________________________________
time_distributed_10 (TimeDis (5, 40, 14, 28, 64)       36928     
_________________________________________________________________
time_distributed_11 (TimeDis (5, 40, 25088)            0         
_________________________________________________________________
lstm_1 (LSTM)                (5, 128)                 

In [27]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError())

history = model.fit(x, y, epochs=10)

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.9031
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 0.9193
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 0.9033
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 0.9041
Epoch 5/10
1/1 [==============================] - 3s 3s/step - loss: 0.9104
Epoch 6/10
1/1 [==============================] - 3s 3s/step - loss: 0.9068
Epoch 7/10
1/1 [==============================] - 3s 3s/step - loss: 0.9017
Epoch 8/10
1/1 [==============================] - 3s 3s/step - loss: 0.9022
Epoch 9/10
1/1 [==============================] - 3s 3s/step - loss: 0.9055
Epoch 10/10
1/1 [==============================] - 3s 3s/step - loss: 0.9058


In [30]:
model.predict(x)

array([[-3.161907],
       [-3.161907],
       [-3.161907],
       [-3.161907],
       [-3.161907]], dtype=float32)

In [29]:
y

array([-3.58, -4.91, -2.49, -2.63, -2.42])

#### Summary
* Current notebook uses 5 samples to train and does not have a test set (from test-1)
* Current model uses the largest force value during an event as the target value
* Current model uses 40 picture frames as the length of an event
* Current model uses simple CNN layers and only a LSTM layer as the RNN part (see model.summary above)

#### Next Steps
* Prepare more data for the training and testing test
* Change model structure